# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv(r'C:\Users\chadhrr\OneDrive - OTPP\Desktop\DSI\scaling_to_production\05_src\data\adult\adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
adult_dt


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,0
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,1
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,0


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [4]:
# Create X (features) and Y (target)
X = adult_dt.drop(columns=['income'])  # Drop 'income' column to get features
Y = adult_dt['income']  # Select 'income' column as the target

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)



Shape of X_train: (22792, 14)
Shape of X_test: (9769, 14)
Shape of Y_train: (22792,)
Shape of Y_test: (9769,)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*

The random_state parameter in the train_test_split function controls the randomness of the data splitting process. Setting random_state to a fixed number ensures the same train-test split is produced every time you run the code, making the results reproducible.

The random state is useful for the following:
Consistency: Ensures consistent results across different runs of the code.
Validation: Allows others to reproduce and validate your results.
Transparency: Facilitates clear and transparent experiments and analyses.
Debugging: Simplifies debugging by eliminating random variations in data splits.
Fair Comparisons: Ensures fair and reliable comparisons in model training and hyperparameter tuning.

random_state enhances reproducibility, a fundamental principle in scientific research and machine learning practice, by ensuring that experiments and results can be consistently replicated.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:


![](./images/assignment_2__column_transformer.png)

In [6]:
# Define categorical and numerical columns
categorical_columns = [
    'workclass', 'education', 'marital-status', 'occupation',
    'relationship', 'race', 'sex', 'native-country'
]
numerical_columns = [
    'age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week'
]

# Define preprocessing for numerical features
numerical_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

# Define preprocessing for categorical features
categorical_transformer = Pipeline([
('impute', SimpleImputer(strategy='most_frequent')),
('onehot', OneHotEncoder(handle_unknown='infrequent_if_exist', drop='if_binary')),
])

# Combine preprocessing for numerical and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])

# Create the final pipeline with preprocessor and logistic regression model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

# Split the data into features (X) and target (y)
X = adult_dt.drop('income', axis=1)
y = adult_dt['income']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
pipeline.fit(X_train, y_train)

# Make predictions on the test set
y_pred = pipeline.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')


Accuracy: 0.86


c:\Users\chadhrr\AppData\Local\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [7]:
from sklearn.ensemble import RandomForestClassifier

# Create the final pipeline with preprocessor and random forest classifier
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Train the model
pipeline.fit(X_train, y_train)

# Make predictions on the test set
y_pred = pipeline.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')


Accuracy: 0.86


# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [10]:

from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score, balanced_accuracy_score, roc_auc_score, log_loss
# Define the scoring metrics
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'balanced_accuracy': make_scorer(balanced_accuracy_score),
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True),
    'neg_log_loss': make_scorer(log_loss, greater_is_better=False, needs_proba=True)
}

# Perform cross-validation
cv_results = cross_validate(pipeline, X, Y, cv=5, scoring=scoring, return_train_score=True)

# Extract and print results
print("Training Scores:")
print("Accuracy:", cv_results['train_accuracy'].mean())
print("Balanced Accuracy:", cv_results['train_balanced_accuracy'].mean())
print("ROC AUC:", cv_results['train_roc_auc'].mean())
print("Negative Log Loss:", cv_results['train_neg_log_loss'].mean())

print("\nValidation Scores:")
print("Accuracy:", cv_results['test_accuracy'].mean())
print("Balanced Accuracy:", cv_results['test_balanced_accuracy'].mean())
print("ROC AUC:", cv_results['test_roc_auc'].mean())
print("Negative Log Loss:", cv_results['test_neg_log_loss'].mean())

c:\Users\chadhrr\AppData\Local\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py:548: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(
c:\Users\chadhrr\AppData\Local\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\chadhrr\AppData\Local\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Training Scores:
Accuracy: 0.9999692883535592
Balanced Accuracy: 0.9999688862195363
ROC AUC: 0.9999999072990683
Negative Log Loss: -0.08059573554539697

Validation Scores:
Accuracy: 0.8547035111406368
Balanced Accuracy: 0.7753846601663925
ROC AUC: 0.9023234397018568
Negative Log Loss: -0.37513430734803066


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [11]:
# Extract fold-level results
fold_results = pd.DataFrame({
    'Train Accuracy': cv_results['train_accuracy'],
    'Train Balanced Accuracy': cv_results['train_balanced_accuracy'],
    'Train ROC AUC': cv_results['train_roc_auc'],
    'Train Neg Log Loss': cv_results['train_neg_log_loss'],
    'Test Accuracy': cv_results['test_accuracy'],
    'Test Balanced Accuracy': cv_results['test_balanced_accuracy'],
    'Test ROC AUC': cv_results['test_roc_auc'],
    'Test Neg Log Loss': cv_results['test_neg_log_loss']
})

# Sort by negative log loss of the test (validation) set
fold_results_sorted = fold_results.sort_values(by='Test Neg Log Loss')

# Display the sorted results
print("Fold-level results sorted by Test Neg Log Loss:")
print(fold_results_sorted)


Fold-level results sorted by Test Neg Log Loss:
   Train Accuracy  Train Balanced Accuracy  Train ROC AUC  Train Neg Log Loss  \
1        0.999962                 0.999975            1.0           -0.079882   
0        0.999962                 0.999920            1.0           -0.080468   
4        0.999962                 0.999975            1.0           -0.081341   
2        0.999962                 0.999975            1.0           -0.080695   
3        1.000000                 1.000000            1.0           -0.080591   

   Test Accuracy  Test Balanced Accuracy  Test ROC AUC  Test Neg Log Loss  
1       0.851966                0.769466      0.896845          -0.395460  
0       0.853677                0.770268      0.902153          -0.379042  
4       0.857647                0.782353      0.906950          -0.375717  
2       0.852119                0.778277      0.902365          -0.369435  
3       0.858108                0.776560      0.903304          -0.356018  


Calculate the mean of each metric. 

In [12]:
# Calculate the mean of each metric
mean_metrics = fold_results_sorted.mean()

# Print the mean metrics
print("Mean of each metric:")
print(mean_metrics)


Mean of each metric:
Train Accuracy             0.999969
Train Balanced Accuracy    0.999969
Train ROC AUC              1.000000
Train Neg Log Loss        -0.080596
Test Accuracy              0.854704
Test Balanced Accuracy     0.775385
Test ROC AUC               0.902323
Test Neg Log Loss         -0.375134
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [14]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Split the data into training and testing sets
X = adult_dt.drop('income', axis=1)
y = adult_dt['income']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Make predictions on the test data
y_pred = pipeline.predict(X_test)
y_pred_proba = pipeline.predict_proba(X_test)

# Calculate performance metrics
metrics = {
    'Test Accuracy': accuracy_score(y_test, y_pred),
    'Test Balanced Accuracy': balanced_accuracy_score(y_test, y_pred),
    'Test ROC AUC': roc_auc_score(y_test, y_pred_proba[:, 1]),
    'Test Neg Log Loss': log_loss(y_test, y_pred_proba)
}

# Display the results as a dictionary
print("Performance metrics on the test data:")
print(metrics)


Performance metrics on the test data:
{'Test Accuracy': 0.8587440503608168, 'Test Balanced Accuracy': 0.7827419324508023, 'Test ROC AUC': 0.9064493123414291, 'Test Neg Log Loss': 0.3764789093489895}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)
Recoding the target variable "income" into a binary format (0 and 1) immediately after loading the data streamlines the data preprocessing workflow, ensures compatibility with machine learning libraries, and simplifies the application of evaluation metrics for binary classification tasks. It's a good practice to align your data format early in the pipeline to optimize the modeling process.

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Evaluation of model pipeline |Model pipeline was evaluated correctly.|Model pipeline was not evaluated correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.